In [82]:
import pandas as pd
import numpy as np
from pyphy import pyphy

In [68]:
virushostdb = pd.read_csv('virushostdb/virushostdb.tsv', sep="\t")
virushostdb.head()

,virus tax id,virus name,virus lineage,refseq id,KEGG GENOME,KEGG DISEASE,DISEASE,host tax id,host name,host lineage,pmid,evidence,sample type,source organism
0,438782,Abaca bunchy top virus,Viruses; Monodnaviria; Shotokuvirae; Cressdnav...,"NC_010314, NC_010315, NC_010316, NC_010317, NC...",NaN,NaN,NaN,214697.0,Musa acuminata AAA Group,Eukaryota; Viridiplantae; Streptophyta; Strept...,17978886,Literature,NaN,NaN
1,1241371,Abalone herpesvirus Victoria/AUS/2009,Viruses; Duplodnaviria; Heunggongvirae; Peplov...,NC_018874,NaN,NaN,NaN,36095.0,Haliotis diversicolor,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,28962967,Literature,NaN,NaN
2,491893,Abalone shriveling syndrome-associated virus,Viruses; Duplodnaviria; Heunggongvirae; Urovir...,NC_011646,NaN,NaN,NaN,37770.0,Haliotis diversicolor aquatilis,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,21079776,"Literature, RefSeq",NaN,NaN
3,11788,Abelson murine leukemia virus,Viruses; Riboviria; Pararnavirae; Artvervirico...,NC_001499,NaN,NaN,NaN,10090.0,Mus musculus,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,6257398,"Literature, UniProt",NaN,NaN
4,2025595,Abisko virus,Viruses; Riboviria; unclassified Riboviria,NC_035470,NaN,NaN,NaN,201501.0,Epirrita autumnata,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,NaN,"Literature, NCBI Virus, RefSeq",NaN,NaN


In [69]:
virus_host_connection = virushostdb[virushostdb["host tax id"].notnull()][["virus tax id", "host tax id"]]

virus_host_connection["host tax id"] = virus_host_connection["host tax id"].astype("int32")

In [70]:
taxon = pd.read_csv("import/taxon.csv")

taxon.head()

,taxid,name,rank
0,10239,Viruses,superkingdom
1,2559587,Riboviria,clade
2,2732409,Artverviricota,phylum
3,2732514,Revtraviricetes,class
4,2732515,Blubervirales,order


In [71]:
len(virus_host_connection[~virus_host_connection["virus tax id"].isin(taxon["taxid"].unique())]["virus tax id"].unique())

0

In [72]:
len(virus_host_connection)

18123

In [73]:
virus_host_connection.to_csv("import/virus_host_connection.csv", index=False)

In [74]:
virus_host_connection.head()

,virus tax id,host tax id
0,438782,214697
1,1241371,36095
2,491893,37770
3,11788,10090
4,2025595,201501


In [76]:
taxon["taxid"].unique()

array([  10239, 2559587, 2732409, ..., 1176434, 1176764, 1176765],
      dtype=int64)

In [79]:
virus_host_connection[~virus_host_connection["host tax id"].isin(taxon["taxid"].unique())]

,virus tax id,host tax id
0,438782,214697
1,1241371,36095
2,491893,37770
3,11788,10090
4,2025595,201501
...,...,...
18120,12232,3663
18121,12232,3670
18122,253701,3594
18123,114871,4932


In [83]:
desired_rank = ["superkingdom", "clade", "phylum", "class", "order", "family", "genus", "species", "genome"]

parent_son = set()

taxid_taxon = {}

for taxid in virus_host_connection[~virus_host_connection["host tax id"].isin(taxon["taxid"].unique())]["host tax id"].unique():
    
    dict_path = pyphy.getDictPathByTaxid(taxid)
    dict_path["genome"] = taxid
    quartett = [""] * 2
    for rank in desired_rank:
        if rank in dict_path:
            
            name = pyphy.getNameByTaxid(dict_path[rank])

            if rank == "superkingdom":

                quartett[0] = dict_path[rank]

                if dict_path[rank] not in taxid_taxon:
                    taxid_taxon[dict_path[rank]] = [name, rank]
            else:
                quartett[1] = dict_path[rank]

                if quartett[0] != quartett[1]:
                    parent_son.add(tuple(quartett))

                    if dict_path[rank] not in taxid_taxon:
                        taxid_taxon[dict_path[rank]] = [name, rank]

                    quartett[0] = quartett[1]

In [84]:
with open("import/host_taxon.csv", "w") as output:
    output.write(",".join(["taxid", "name", "rank"]) + "\n")

for taxid in taxid_taxon:
    with open("import/host_taxon.csv", "a") as output:
        output.write(",".join([f'"{taxid}"', f'"{taxid_taxon[taxid][0]}"', f'"{taxid_taxon[taxid][1]}"']) + "\n")

In [85]:
with open("import/host_taxon_connections.csv", "w") as output:
    output.write(",".join(["from", "to"]) + "\n")

for parent in parent_son:
    with open("import/host_taxon_connections.csv", "a") as output:
        #["~id", "~from", "~to", "~label"]
        output.write(",".join([f'"{parent[0]}"', f'"{parent[1]}"']) + "\n")